# Pandas: Conociendo la biblioteca

## Temas
- La biblioteca pandas
- Limpieza, manipulacion, analisis y visualizacion de datos
- Explorar las caracteristicas de una base
- Realizar un analisis exploratorio
- Lidiar con valores nulos, remover registros, crear columnas
- Aplicar filtros, entre otras operaciones...

## **Pandas**

### Importando la base de datos

In [ ]:
import pandas as pd

url = 'https://gist.githubusercontent.com/ahcamachod/a572cfcc2527046db93101f88011b26e/raw/ffb13f45a79d31223e645611a119397dd127ee3c/alquiler.csv'

pd.read_csv('alquiler.csv', sep=';')
practica = pd.read_csv('alquiler.csv', sep=';')

In [ ]:
datos = pd.read_csv('alquiler.csv', sep=';')#-> almaceno los registros en una variable

In [ ]:
datos.head()#->encabezado con los primeros 5 registros

In [ ]:
datos.head(10)#->encabezado con los primeros 10 registros

In [ ]:
datos.tail()#->retorna los ultimos 5 registros del csv

In [ ]:
type(datos)#->pandas.core.frame.DataFrame

### Explorar las caracteristicas generales de los datos

In [ ]:
datos.shape #(25121, 9) -> cant filas y columnas

In [ ]:
datos.columns#Index(['Tipo', 'Colonia', 'Habitaciones', 'Garages', 'Suites', 'Area', 'Valor', 'Condominio', 'Impuesto'], dtype='object')

In [ ]:
datos.info()

In [ ]:
datos["Tipo"]

In [ ]:
datos[['Habitaciones', 'Valor']]

## Pandas - Analisis exploratorio de los datos

- ¿Cuáles son los valores promedio de alquiler por tipo de propiedad?

In [ ]:
datos.head()

In [ ]:
datos["Valor"].mean() #->np.float64(36885.05048392878)

In [ ]:
#agrupar datos por tipo
datos.groupby('Tipo').mean(numeric_only=True)

In [ ]:
#me muestra todos los valores promedio de alquiler por tipo de inmueble
datos.groupby('Tipo')['Valor'].mean()

In [ ]:
#lo muestro como dataframe: [['Valor]] y no como serie y lo organizo por valor
datos.groupby('Tipo')[['Valor']].mean().sort_values('Valor')

In [ ]:
#lo almaceno en una variable para graficarlo con pandas(plot)
df_tipo_precio = datos.groupby('Tipo')[['Valor']].mean().sort_values('Valor')
df_tipo_precio.plot(kind='barh', figsize=(12,8), color='purple')#-> kind('barh'): barras horizontales, figsize=(12,8): tamaño figura, color de las barras

In [ ]:
#lista con los inmuebles comerciales
datos.Tipo.unique()

In [ ]:
inmuebles_comerciales = ['Conjunto Comercial/Sala', 'Edificio Completo', 'Tienda/Salón', 'Casa Comercial','Cochera/Estacionamiento', 'Terreno Estándar', 
                         'Galpón/Depósito/Almacén', 'Tienda en Centro Comercial', 'Hotel', 'Loteo/Condominio', 'Industria']

In [ ]:
#me muestra los inmuebles comerciales que estan en tipo
datos.query('@inmuebles_comerciales in Tipo')

In [ ]:
#me muestra los inmuebles residenciales
df = datos.query('@inmuebles_comerciales not in Tipo')

In [ ]:
#me muestra un grafico de los residenciales
df_tipo_precio = df.groupby('Tipo')[['Valor']].mean().sort_values('Valor')
df_tipo_precio.plot(kind='barh', figsize=(12,8), color='purple')

### Porcentaje de cada tipo de inmueble presente en la base de datos

In [ ]:
df.Tipo.unique()

In [ ]:
df.Tipo.value_counts()

In [ ]:
df.Tipo.value_counts(normalize=True)

In [ ]:
porcentaje_tipo = df.Tipo.value_counts(normalize=True).to_frame()
porcentaje_tipo.plot(kind='bar', figsize=(12,8), color="green", xlabel='Tipos', ylabel='Porcentaje')

In [ ]:
# me centro en el tipo departamento que es el que tiene mayor cantidad de insidencia
df = df.query('Tipo == "Departamento"')

In [ ]:
df.shape#-> porcentaje

#### Desafio

In [ ]:
#1-Calcular el promedio de habitaciones por departamento.
df['Habitaciones'].mean() #-> np.float64(2.5041994039555675)

In [ ]:
#2-Verificar cuántas colonias únicas existen en nuestra base de datos.
len(df['Colonia'].unique())

In [ ]:
#3-Analizar qué colonias tienen el promedio de alquiler más alto.
#me muestra todos los valores promedio de alquiler por tipo de inmueble
#datos.groupby('Tipo')['Valor'].mean()
df.groupby('Colonia')[['Valor']].mean().sort_values('Valor', ascending=False)

In [ ]:
#4-Crear un gráfico de barras horizontales que muestre las 5 colonias con los promedios de alquiler más altos.
prom_colonias = df.groupby('Colonia')[['Valor']].mean().sort_values('Valor', ascending=False).head()
prom_colonias.plot(kind='barh', figsize=(11,6), color='blue', xlabel='Valor', ylabel='Colonia')

# 03 - Tratamiento y filtrado de los datos

- crear un modelo para estimar el precio de alquiler de las propiedades, es importante que tratemos los datos faltantes,   
  ya que los datos nulos no pueden utilizarse en el entrenamiento de modelos de ML

In [ ]:
#verificar 1- la cantidad de datos nulos y  2- tratarlos

df.isnull()#->me muestra la cantidad de datos nulos por cada columna

In [ ]:
# 1-
df.isnull().sum()#-> cada columna con la cantidad de nulos

In [ ]:
# 2-
#tratar los datos nulos y mostrarlo como 0(cero)
df = df.fillna(0)
df.isnull().sum()

Algunos registros en la base de datos no tienen mucho sentido, por ejemplo:  

- 1 Departamentos con un valor de alquiler igual a 0.  
- 2 Departamentos con un valor de condominio igual a 0.  

Estos registros son inconsistentes, por lo tanto, debemos eliminarlos de nuestra base de datos.  

In [ ]:
#1
df.query('Valor == 0 | Condominio == 0')

In [ ]:
#con index traigo todos los indices que contienen valor 0 o condominio 0
df.query('Valor == 0 | Condominio == 0').index

In [ ]:
#elimino todos los registros con esos indices
#aca creo el filtro con los registros a remover
df_remover = df.query('Valor == 0 | Condominio == 0').index

In [ ]:
#con drop remuevo esos indices almacenados, y le indico con axis= 0(filas) - 1(columnas), 
# inplace le indico que lo aplique en el dataframe sin necesidad de almacenarlo en una variable
df.drop(df_remover, axis=0, inplace=True)#-> punto 1 realizado

In [ ]:
df.head()#->

In [ ]:
df.drop('Tipo', axis=1, inplace=True)#-> elimino la columna(axis=1) Tipo, ya que todos son departamentos
df.sample(10)#-> me muestra 10 registros

En este momento, el equipo de ML desea evaluar solo propiedades en escenarios específicos, como:

- 1- Apartamentos que tienen 1 dormitorio y un alquiler menor a MXN 4200.
- 2- Apartamentos que tienen al menos 2 dormitorios, un alquiler menor a MXN 10500 y una superficie mayor a 70 m².

Con base en esto, necesitamos aplicar los filtros necesarios para obtener esta información específica.

In [ ]:
#1
seleccion1 = df['Habitaciones'] == 1
df[seleccion1]#-> me retorna los departamentos que tiene solo 1 habitacion

In [ ]:
seleccion2 = df['Valor'] < 4200 #-> el valor del departamento sea menor a 4200
df[seleccion2]

In [ ]:
#1
filtro1 = (seleccion1) & (seleccion2) #-> almaceno los dos filtros
df1 = df[filtro1] #-> aplico el filtro al dataframe

In [ ]:
#2
filtro2 = (df['Habitaciones'] >= 2) & (df['Valor'] < 10500) & (df['Area'] > 70)
df2 = df[filtro2]

Para finalizar, necesitamos almacenar nuestros dataframes en formato csv, tras realizar todos los análisis y tratamientos.

- 1- Almacenar los dataframe(s) completos tras realizar todas las modificaciones

In [ ]:
df.to_csv('inmuebles_ml.csv', index=False, sep=';')#--> le indico que quede guardado con separador de ;

In [ ]:
pd.read_csv('inmuebles_ml.csv', sep=';')

In [ ]:
df1.to_csv('inmuebles_ml_filtro1.csv', index=False, sep=';')
df2.to_csv('inmuebles_ml_filtro2.csv', index=False, sep=';')

# 04 - Manipulacion de los datos

* El equipo de desarrollo del sitio web de la empresa solicitó la creación de dos nuevas columnas numéricas en nuestra base de datos. En el sitio web de las propiedades, desean presentar información más resumida en relación a los valores. Para esto, estas columnas serán importantes:

 - 1 valor_mensual: Esta columna debe contener los gastos mensuales de cada propiedad, incluyendo el alquiler y el condominio.

 - 2 valor_anual: Esta columna debe contener los gastos anuales por propiedad, es decir, el Impuesto más 12 meses de alquiler y condominio.

In [ ]:
#1
import pandas as pd
import numpy as np
datos = pd.read_csv('alquiler.csv', sep=';')

In [ ]:
#1
datos['Valor_mensual'] = datos['Valor'] + datos['Condominio']
datos.head()

In [ ]:
#2
datos['Valor_anual'] = datos['Valor_mensual'] * 12 + datos['Impuesto']
datos.head()

El equipo de desarrollo solicitó la creación de dos columnas adicionales. Sin embargo, esta vez son categóricas:

- 1 Descripcion: Esta columna debe contener un resumen de la información clave de las propiedades que se mostrarán en el sitio web: tipo de propiedad, barrio, cantidad de habitaciones y plazas de estacionamiento.

- 2 Tiene_suite: Esta debe ser una columna que indique únicamente si la propiedad tiene o no suites, sin importar la cantidad.

In [ ]:
#1
datos['Descripcion'] = datos['Tipo'] + " en " + datos['Colonia'] + " con " + datos['Habitaciones'].astype(str) + \
                    " habitacion(es) y " + np.where(datos['Garages'] < 1, "Sin estacionamiento", datos['Garages'].astype(str) + "estacionamiento(s)")

In [ ]:
datos.iloc[500].Descripcion #-> 'Departamento en Napoles con 1 habitacion(es) y Sin estacionamiento'

In [ ]:
#2
datos['Tiene_suite'] = np.where(datos['Suites'] < 1, "No tiene", "Tiene")
#otra forma
#datos['Tiene_suite'] = datos['Suites'].apply(lambda x: 'Si' if x > 0 else 'No')

In [ ]:
datos.to_csv('inmuebles_dev.csv', index=False, sep=';')